Celem pliku jest umieszczenie próbek zebranych od wolontariuszy w folderze zawierającym dwa foldery, jeden na klasę. Aby program działał potrzeba struktura plików to:


```bash
.
├── people_images/ <= starting structure
│   ├── first_human/
│   │   ├── krzywo.zip
│   │   └── prosto.zip
│   ├── second_human/
│   │   ├── krzywo.zip
│   │   └── prosto.zip
│   ├── third_human/
│   │   ├── krzywo.zip
│   │   └── prosto.zip
│   └── ...
└── classifided_images/ <= desired
    ├── neg-straight-class
    └── pos-nonstraight-class
```

Wymagania:
1. Utworzony folder docelowy, zawierający dwa foldery, jeden na klasę. Folder z klasą "nieprosto" (pozytywny) ma nazwę alfabetycznie drugą. Wynika to z tego jak działa funkcja keras.utils.image_dataset_from_directory
2. Utworzony folder źródłowy zawierający folder na osobę a w każdym z nich dwa foldery/zipy ze zdjęciami na klasę. **Kluczowe jest by każdy folder zawierający klasę pozytywną miał pewne słowo, tak samo jak każdy folder z klasą negatywną. Słowa muszą być różne.** Przykład widoczny na strukturze.

In [1]:
import os
import shutil

Ustawiamy:
1. Ścieżkę folderu źródłowego
2. Ścieżki folderów docelowych.
3. Słowa za pomocą których rozróżnimy jedną i drugą klasę.

In [3]:
samples_path = "./people_images" 

base_destination_path = "./classified_images"
slouching_samples_path = os.path.join(base_destination_path, "pos-nonstraight-class")
straight_samples_path = os.path.join(base_destination_path, "neg-straight-class")

nonstraight_images_folders_contain="krzywo"
straight_images_folders_contain="prosto"

Poniższy blok użyj wyłącznie jeśli zdjęcia są w .zip! Jeśli są już w zwykłych folderach, uruchom tylko kolejny blok.
</br> Uwaga! **Kod domyślnie usuwa foldery zip!**

In [3]:
#Here we simply unzip recursively all folders, and if opted, delete zipped ones.
def unzip_nested_zip_dirs(dir_path, do_remove=False):
    for file in os.scandir(dir_path):
        if(file.is_dir()): unzip_nested_zip_dirs(file.path, True)
        elif(file.name.endswith(".zip")): 
            shutil.unpack_archive(file.path, file.path.removesuffix(".zip"))
            if(do_remove): os.remove(file.path)


unzip_nested_zip_dirs(samples_path, do_remove = True)

Poniższy kod przemieszcza zdjęcia do użytecznej struktury

In [8]:
# This function is required in function definied below, in order to quickly understand skipping is recommended.
#Images in every catalog are named as numbers. (0.jpg, 1.jpg etc.) Therefore in order to move them into
#one common directory (have to have unique filenames), renaming is required, for example, by adding prefix.
def move_all_files(source_folder_path, destination_folder_path, prefix):
    new_filepath_prefix = destination_folder_path if not prefix else os.path.join(destination_folder_path, prefix)
    for file in os.scandir(source_folder_path):
        new_filepath = new_filepath_prefix+file.name 
        shutil.copy(file.path, new_filepath)
        
#Function recursively finds all images amongst directories and move them into either "krzywo" or "prosto" class folder 
#Since all folders contain either "krzywo" or "prosto" in filename by this way program distinguish proper class of images. 
def sort_people_images_out_into_two_classes(dir_path):
    for file in os.scandir(dir_path):
        if(file.is_dir()): 
            dir_name = file.name.lower()
            grand_parent_dir_name = os.path.basename(os.path.dirname(file.path))
            trait = dir_name+"_"+grand_parent_dir_name
            if(nonstraight_images_folders_contain in dir_name): 
                move_all_files(file.path, slouching_samples_path, prefix=trait)
            elif(straight_images_folders_contain in dir_name): 
                move_all_files(file.path, straight_samples_path, prefix=trait)
            else:
                sort_people_images_out_into_two_classes(file.path)

sort_people_images_out_into_two_classes(samples_path)